In [1]:
import copy
import random
import os
import time
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from torchvision import transforms
from torchvision.models import vgg11
from torch.utils.data import Dataset, DataLoader,random_split, SubsetRandomSampler, WeightedRandomSampler
import numpy as np
from sklearn.metrics import f1_score

from focal_loss import FocalLoss
from label_smoothing import LabelSmoothingLoss
from efficientnet_pytorch import EfficientNet
from dataset import TrainDataset, TestDataset, img_transform, TrainDatasetAgeAugmentation
from avgMeter import AverageMeter
from acc_per_label import AccPerLabel

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)



In [2]:
transform = img_transform()


In [3]:
train_root = '/opt/ml/input/data/train/images'
test_root = '/opt/ml/input/data/eval'


In [4]:
batch_size = 128
lr = 0.0001
num_epochs = 15
model_name = 'effnetb0-batchsize_' + str(batch_size) + '-lr_' + str(lr).split('.')[1] + '-epoch_' + str(num_epochs)  + '-cosin_warm_restart' \
                + '-nofreeze' + '-fix_data' + '-no_overlap' + '-age_aug58' + '-Hflip05' + '-center_crop500250' + '-LS_alpha06'
log_dir = '/opt/ml/code/log/' + model_name + '.txt' 
save_dir = '/opt/ml/code/trained_models/' + model_name + '.pt'

In [5]:
# data = TrainDataset(train_root, input_size = 224, transform = transform)
data = TrainDatasetAgeAugmentation(train_root, input_size = 224, transform = transform)

In [6]:
indices = list(range(len(data)))
split_idx = int(len(data) * 0.8)
train_idx, valid_idx = indices[:split_idx], indices[split_idx:]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(data, batch_size=batch_size, num_workers = 3, sampler=train_sampler, pin_memory=True, shuffle=False)
valid_loader = DataLoader(data, batch_size=batch_size, num_workers = 3, sampler=valid_sampler, pin_memory=True, shuffle=False)

In [7]:
# num_class = 18

# indices = list(range(len(data)))
# split_idx = int(len(data) * 0.8)
# train_idx, valid_idx = indices[:split_idx], indices[split_idx:]

# ##################
# from collections import Counter
# labels = data.label_list[:split_idx]
# class_weight = Counter(labels)
# for i in range(len(class_weight)):
#     class_weight[i] = 100.0 / class_weight[i]

# class_weight = torch.Tensor([class_weight[i] for i in range(num_class)])
# # class_weight = torch.Tensor([1/0.97, 1/0.69, 1/0.67, 1/0.95, 1/0.83, 1/0.51, 1/0.94, 1/0.63, 1/0.76, 1/0.94 \
# #                             , 1/0.8, 1/0.62, 1/0.96, 1/0.71, 1/0.73, 1/0.95, 1/0.8, 1/0.55]) - 0.4

# # class_weight = torch.Tensor([1/0.97, 1/0.6, 1/0.67, 1/0.95, 1/0.51, 1/0.51, 1/0.94, 1/0.63, 1/0.76, 1/0.94 \
# #                             , 1/0.6, 1/0.62, 1/0.96, 1/0.6, 1/0.73, 1/0.95, 1/0.6, 1/0.55]) - 0.6
# class_weight_all = class_weight[torch.Tensor(labels).long()]

# train_sampler = WeightedRandomSampler(
#     weights=class_weight_all,
#     num_samples=len(class_weight_all),
#     replacement=True
# )
# ######################
# valid_sampler = SubsetRandomSampler(valid_idx)

# train_loader = DataLoader(data, batch_size=batch_size, num_workers = 4, sampler=train_sampler, pin_memory=True, shuffle=False)
# valid_loader = DataLoader(data, batch_size=batch_size, num_workers = 4, sampler=valid_sampler, pin_memory=True, shuffle=False)

In [8]:
# train, val = random_split(data, [int(len(data)*0.8), int(len(data)*0.2)])
# train_loader = DataLoader(train, batch_size=batch_size, num_workers = 4,  pin_memory=True, shuffle=True)
# valid_loader = DataLoader(val, batch_size=batch_size, num_workers = 4,  pin_memory=True, shuffle=False)

In [9]:
print(model_name)

effnetb0-batchsize_128-lr_0001-epoch_15-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-center_crop500250-LS_alpha06


In [10]:
print(save_dir)

/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_15-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-center_crop500250-LS_alpha06.pt


In [11]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=18)

Loaded pretrained weights for efficientnet-b0


In [12]:
# model._fc = nn.Linear(in_features=1280, out_features=18, bias=True)
model.cuda()
print('haha')

haha


In [13]:
# for n, p in model.named_parameters():
#     if '_fc' not in n:
#         p.requires_grad = False

In [14]:
# criterion = nn.CrossEntropyLoss()
# criterion = FocalLoss(gamma = 100)
criterion = LabelSmoothingLoss(classes=18, smoothing=0.6)
optimizer = Adam(model.parameters(), lr=lr)

scheduler = CosineAnnealingWarmRestarts(optimizer, 5, 1)
# scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.1, patience = 5)

In [15]:
start = time.time()

best_f1 = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
train_loss, train_acc = AverageMeter(), AverageMeter()
valid_loss, valid_acc = AverageMeter(), AverageMeter()
f1 = AverageMeter()
accs = AccPerLabel(num_class = 18)
with open(log_dir, 'w') as log:
    for epoch in range(num_epochs):
        train_loss.reset()
        train_acc.reset()
        for iter, (img, label) in enumerate(train_loader):
            optimizer.zero_grad()

            img, label = img.float().cuda(), label.cuda()

            pred_logit = model(img)

            loss = criterion(pred_logit, label)

            loss.backward()
            optimizer.step()
            scheduler.step(epoch + iter/len(train_loader))
            
            pred_label = pred_logit.argmax(-1)
            acc = (pred_label == label).sum().float() / img.size(0)

            train_loss.update(loss.item(), len(img))
            train_acc.update(acc, len(img))
        
            
            
        valid_loss.reset()
        valid_acc.reset()
        f1.reset()
        accs.reset()
        for img, label in valid_loader:
            img, label = img.float().cuda(), label.cuda()

            with torch.no_grad():
                pred_logit = model(img)


            loss = criterion(pred_logit, label)

            pred_label = pred_logit.argmax(-1)
            acc = (pred_label == label).sum().float() / img.size(0)
            
            valid_loss.update(loss.item(), len(img))
            valid_acc.update(acc, len(img))
            pred_label = pred_label.cpu().numpy()
            label = label.cpu().numpy()
            
            accs.update(pred_label, label)
            f1.update(f1_score(pred_label, label, average='macro'), len(img))
            
        train_loss_val = train_loss.avg
        train_acc_val = train_acc.avg
        valid_loss_val = valid_loss.avg
        valid_acc_val = valid_acc.avg
        f1_val = f1.avg
        
        print("epoch [%3d/%3d] | Train Loss %.4f | Train Acc %.4f | Valid Loss %.4f | Valid Acc %.4f | f1_score %.4f" %
            (epoch+1, num_epochs, train_loss_val, train_acc_val, valid_loss_val, valid_acc_val, f1_val))
        
        accs_result = accs.show_result()
        print(accs_result)
        if f1_val > best_f1:
            best_f1 = f1_val
            best_model_wts = copy.deepcopy(model.state_dict())
        
        
        # Train Log Writing
        log.write("epoch [%3d/%3d] | Train Loss %.4f | Train Acc %.4f | Valid Loss %.4f | Valid Acc %.4f | f1_score %.4f\n" %
            (epoch+1, num_epochs, train_loss_val, train_acc_val, valid_loss_val, valid_acc_val, f1_val))
        log.write(accs_result + '\n')
    print(f"training time : {(time.time() - start)/60}")    
    log.write(f"training time : {(time.time() - start)/60}")

epoch [  1/ 15] | Train Loss 2.6173 | Train Acc 0.6466 | Valid Loss 2.5282 | Valid Acc 0.7831 | f1_score 0.5873
class_number num_true_positive/num_class_i = acc
0   505.0/515= 0.98|| 1   252.0/390= 0.65|| 2   100.0/165= 0.61|| 3   691.0/730= 0.95|| 4   597.0/690= 0.87|| 5   93.0/210= 0.44|| 6   90.0/103= 0.87|| 7   16.0/78= 0.21|| 8   0.0/33= 0.00|| 
9   128.0/146= 0.88|| 10   92.0/138= 0.67|| 11   1.0/42= 0.02|| 12   100.0/103= 0.97|| 13   32.0/78= 0.41|| 14   1.0/33= 0.03|| 15   138.0/146= 0.95|| 16   124.0/138= 0.90|| 17   0.0/42= 0.00|| 
epoch [  2/ 15] | Train Loss 2.4888 | Train Acc 0.8397 | Valid Loss 2.4931 | Valid Acc 0.8328 | f1_score 0.6959
class_number num_true_positive/num_class_i = acc
0   506.0/515= 0.98|| 1   304.0/390= 0.78|| 2   107.0/165= 0.65|| 3   677.0/730= 0.93|| 4   611.0/690= 0.89|| 5   93.0/210= 0.44|| 6   93.0/103= 0.90|| 7   55.0/78= 0.71|| 8   0.0/33= 0.00|| 
9   136.0/146= 0.93|| 10   123.0/138= 0.89|| 11   7.0/42= 0.17|| 12   99.0/103= 0.96|| 13   60.0/78

In [16]:
model.__class__.__name__

'EfficientNet'

In [17]:
model.load_state_dict(best_model_wts)
torch.save(model, save_dir)